# Time series forecasting with tf

## Tasks

![types](https://discuss.pytorch.org/uploads/default/6415da0424dd66f2f5b134709b92baa59e604c55)

### Task1 
Given 10 timesteps of features, predict the USDT of the next timestep (many to one)

### Task2
Given 10 timestaps of features, predict the USDT of those timesteps (many to many)

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten, Conv1D, SimpleRNN

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

### Load the dataset

In [ ]:
data = pd.read_csv('../../assets/data/cryptocurrency_data.csv')

### Normalizing data

In [ ]:
features = ['high', 'low', 'open', 'close', 'Volume XRP', 'Volume USDT']

0   unix         1334 non-null   float64
 1   date         1334 non-null   object 
 2   symbol       1334 non-null   object 
 3   open         1334 non-null   float64
 4   high         1334 non-null   float64
 5   low          1334 non-null   float64
 6   close        1334 non-null   float64
 7   Volume XRP   1334 non-null   float64
 8   Volume USDT  1334 non-null   float64

## Task1

### Data preparation

Preparing the data for the first task

We will try to predict the value of USDT on the 11th day using the data of other features from day 1 to 10

In [ ]:
def sequential_window_dataset(series, window_size):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=window_size, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(window_size + 1))
    ds = ds.map(lambda window: (window[:-1], window[1:]))
    return ds.batch(1).prefetch(1)

In [ ]:
def data_preparation(df, lookback, future, scaler):
    date_train = pd.to_datetime(df['date'])
    features = ['high', 'low', 'open', 'close', 'Volume XRP', 'Volume USDT']
    df_train = df[features]
    df_train = df_train.astype(float)

    df_train_scaled = scaler.transform(df_train)

    X, y = [], []
    for i in range(lookback, len(df_train_scaled)-future+1):
        X.append(df_train_scaled[i-lookback:i, 0:df_train.shape[1]])
        y.append(df_train_scaled[i+future-1:i+future, 0])

    return np.array(X), np.array(y), df_train, date_train

In [ ]:
def predict(model, date_train, X_train, future_steps):
    dates = pd.date_range(
        list(date_train)[-1], periods=future_steps, freq='1d').tolist()
    predicted = model.predict(X_train[-future_steps:])
    nsamples, nx, ny = predicted.shape
    predicted = predicted.reshape((nsamples, nx * ny))
    return predicted, dates


def output_preparation(forecasting_dates, predictions, date_column='date', predicted_column='Volume USDT'):
    dates = [date.date() for date in forecasting_dates]
    predicted_df = pd.DataFrame(columns=[date_column, predicted_column])
    predicted_df[date_column] = pd.to_datetime(dates)
    predicted_df[predicted_column] = predictions
    return predicted_df

In [ ]:
def results(df, lookback, future, scaler, col, X_train, y_train, df_train, date_train, model):
    predictions, forecasting_dates = predict(
        model, date_train, X_train, future)
    results_df = output_preparation(forecasting_dates, predictions)
    print(results_df.head())
    fig = plt.figure()
    plt.plot(results_df['date'], results_df['Volume USDT'])

In [ ]:
scaler = StandardScaler()
features = ['high', 'low', 'open', 'close', 'Volume XRP', 'Volume USDT']
df_train = data[features]
df_train = df_train.astype(float)

scaler = scaler.fit(df_train)
df_scaled = scaler.transform(df_train)

In [ ]:
scaler_y = StandardScaler()
df_y = scaler_y.fit_transform(data[['Volume USDT']].to_numpy())

In [ ]:
window_size = 10
future = 1
X_train, y_train, df_train, date_train = data_preparation(
    data, window_size, future, scaler)

### Modeling

#### ANN

In [ ]:
def ann_builder(X, y):
    model = tf.keras.models.Sequential()
    model.add(Dense(12, activation='selu',
              input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(y.shape[1]))

    optimizer = tf.keras.optimizers.Adam()
    model.compile(
        loss='huber',
        optimizer=optimizer,
        metrics=['mse']
    )

    model.fit(X, y, epochs=50, verbose=1, validation_split=0.1, batch_size=16)
    return model

In [ ]:
model = ann_builder(X_train, y_train)

In [ ]:
pd.DataFrame(model.history.history).plot()

In [ ]:
results(data, window_size, future, scaler, 'Volume USDT',
        X_train, y_train, df_train, date_train, model)

#### CNN

In [ ]:
def cnn_builder(X, y):
    model = tf.keras.models.Sequential()
    model.add(Conv1D(32, (3), activation='relu',
              input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(y.shape[1]))

    optimizer = tf.keras.optimizers.Adam()
    model.compile(
        loss='huber',
        optimizer=optimizer,
        metrics=['mse']
    )

    model.fit(X, y, epochs=50, verbose=1, validation_split=0.1, batch_size=16)
    return model

In [ ]:
model = cnn_builder(X_train, y_train)

In [ ]:
pd.DataFrame(model.history.history).plot()

In [ ]:
results(data, window_size, future, scaler, 'Volume USDT',
        X_train, y_train, df_train, date_train, model)

#### Simple RNN

In [ ]:
def rnn_builder(X, y):
    model = tf.keras.models.Sequential()
    model.add(SimpleRNN(1, input_shape=(
        X.shape[1], X.shape[2]), return_sequences=True))
    model.add(Dense(y.shape[1]))

    optimizer = tf.keras.optimizers.Adam()
    model.compile(
        loss='huber',
        optimizer=optimizer,
        metrics=['mse']
    )

    model.fit(X, y, epochs=50, verbose=1, validation_split=0.1, batch_size=16)
    return model

In [ ]:
model = rnn_builder(X_train, y_train)

In [ ]:
pd.DataFrame(model.history.history).plot()

In [ ]:
results(data, window_size, future, scaler, 'Volume USDT',
        X_train, y_train, df_train, date_train, model)

#### LSTM

In [ ]:
def lstm_builder(X, y):
    model = tf.keras.models.Sequential()
    model.add(LSTM(1, input_shape=(
        X.shape[1], X.shape[2]), return_sequences=True))
    model.add(Dense(y.shape[1]))

    optimizer = tf.keras.optimizers.Adam()
    model.compile(
        loss='huber',
        optimizer=optimizer,
        metrics=['mse']
    )

    model.fit(X, y, epochs=50, verbose=1, validation_split=0.1, batch_size=16)
    return model

In [ ]:
model = lstm_builder(X_train, y_train)

In [ ]:
results(data, window_size, future, scaler, 'Volume USDT',
        X_train, y_train, df_train, date_train, model)

## Task 2

Just setting future to 10 and the window to 10

In [ ]:
window_size = 20
future = 20
X_train, y_train, df_train, date_train = data_preparation(
    data, window_size, future, scaler)

Same models

### ANN


In [ ]:
def ann_builder(X, y):
    model = tf.keras.models.Sequential()
    model.add(Dense(32, activation='selu',
              input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(y.shape[1]))

    optimizer = tf.keras.optimizers.Adam()
    model.compile(
        loss='huber',
        optimizer=optimizer,
        metrics=['mse']
    )

    model.fit(X, y, epochs=50, verbose=1, validation_split=0.1, batch_size=16)
    return model

In [ ]:
model = ann_builder(X_train, y_train)

In [ ]:
results(data, window_size, future, scaler, 'Volume USDT',
        X_train, y_train, df_train, date_train, model)

### RNN

In [ ]:
def rnn_builder(X, y):
    model = tf.keras.models.Sequential()
    model.add(SimpleRNN(1, input_shape=(
        X.shape[1], X.shape[2]), return_sequences=True))
    model.add(Dense(y.shape[1]))

    optimizer = tf.keras.optimizers.Adam()
    model.compile(
        loss='huber',
        optimizer=optimizer,
        metrics=['mse']
    )

    model.fit(X, y, epochs=50, verbose=1, validation_split=0.1, batch_size=16)
    return model

In [ ]:
model = rnn_builder(X_train, y_train)

In [ ]:
pd.DataFrame(model.history.history).plot()

In [ ]:
results(data, window_size, future, scaler, 'Volume USDT',
        X_train, y_train, df_train, date_train, model)

### CNN

In [ ]:
def cnn_builder(X, y):
    model = tf.keras.models.Sequential()
    model.add(Conv1D(32, (3), activation='relu',
              input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(y.shape[1]))

    optimizer = tf.keras.optimizers.Adam()
    model.compile(
        loss='huber',
        optimizer=optimizer,
        metrics=['mse']
    )

    model.fit(X, y, epochs=50, verbose=1, validation_split=0.1, batch_size=16)
    return model

In [ ]:
model = cnn_builder(X_train, y_train)

In [ ]:
pd.DataFrame(model.history.history).plot()

In [ ]:
results(data, window_size, future, scaler, 'Volume USDT',
        X_train, y_train, df_train, date_train, model)

### LSTM


In [ ]:
def lstm_builder(X, y):
    model = tf.keras.models.Sequential()
    model.add(LSTM(1, input_shape=(
        X.shape[1], X.shape[2]), return_sequences=True))
    model.add(Dense(y.shape[1]))

    optimizer = tf.keras.optimizers.Adam()
    model.compile(
        loss='huber',
        optimizer=optimizer,
        metrics=['mse']
    )

    model.fit(X, y, epochs=50, verbose=1, validation_split=0.1, batch_size=16)
    return model

In [ ]:
model = lstm_builder(X_train, y_train)

In [ ]:
pd.DataFrame(model.history.history).plot()

In [ ]:
results(data, window_size, future, scaler, 'Volume USDT',
        X_train, y_train, df_train, date_train, model)

## Acknowledgments
Thanks to Taha Bouhsine for creating the Notebook [WTS2: Time Series Forecasting with TF](https://www.kaggle.com/code/skywolfmo/wts2-time-series-forecasting-with-tf/notebook), lisensed under the [Apache 2.0](https://www.apache.org/licenses/LICENSE-2.0). It inspires the majority of the content in this chapter.